In [ ]:
from typing import *

# Condition Filter
Thus far, the user searched for matches in the input graph that would match one of the LHS patterns he passed as parameter, and the matches found were converted to a ResultSet. 

Before these matches are passed to the transformation phase of rewrite, the user can filter matches based on a **Condition** function, which receives a Match instance, checks its attributes and returns a boolean value. 
If the condition function returned False for some pair, this match is then filtered by rewrite and will not be transformed in the next phase.

In [ ]:
ConditionFunc = Callable[Match, bool]

NameError: name 'Match' is not defined

### Filter Matches based on a Condition function
Receives a ResultSet object and a Condition function, and returns a filtered ResultSet.

In [ ]:
def filter_matches(unfiltered_results: ResultSet, condition: ConditionFunc) -> ResultSet:
    # initialize new_matches, empty list

    # for match in unfiltered_results.matches:
    #   if condition(match):
    #       add match to new_matches

    # return ResultSet(new_matches)
    pass